In [ ]:
import torch
from vit import ViT



v = ViT(
    transformer_layers = 2,
    image_size = 256,
    patch_size = 32,
    num_classes = 1000,
    dim = 1024,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)

img = torch.randn(1, 3, 256, 256)

preds = v(img) # (1, 1000)

In [4]:
import torch
import torch.nn as nn

m = nn.Identity(54, unused_argument1=0.1, unused_argument2=False)
input = torch.randn(128, 20)
output = m(input)

assert torch.equal(input, output), "Identity module should not modify the input tensor"
print("Identity module works correctly")

Identity module works correctly
